In [ ]:
import os
import pandas as pd
import yaml
from utility import *

In [ ]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join 
    
model_outputs_dir = params['model_dir']
s2s_dir = _join(model_outputs_dir, "output_summaries")
concept_id = params['concept_id']
perf_measure_columns = params['final_columns']
filename_extension = params['filename_extension']
summary_outputs = params['summary_dir']

In [ ]:
TOD = ["ea", "am", "md", "pm", "ev"]
mode = ["KNR_TRN_WLK", "WLK_TRN_KNR", "PNR_TRN_WLK", "WLK_TRN_PNR", "WLK_TRN_WLK"]

rail = pd.DataFrame(columns=["board_st", "alight_st", "count", "TOD", "mode"]) 

for t in TOD:
    for m in mode:
        file_path = _join(s2s_dir, f"all rail_station_to_station_{m}_{t}.txt")
        fwf_positions = [(2, 8), (11, 17), (20, 28)]
        col_names = ["board_st", "alight_st", "count"]

        file = pd.read_fwf(file_path, colspecs=fwf_positions, header=None, names=col_names, skiprows=18)
        
        file['count'] = file['count'].astype('str')
        file = file[file['count'].str.contains('u')==False]
        file = file[file['count'].str.contains('n')==False]
        file['count'] = pd.to_numeric(file['count'])
        
        file["TOD"] = t
        file["mode"] = m

        rail = pd.concat([rail, file], ignore_index=True)

#print(rail)

In [ ]:
rail_df = summarize_all_combinations(rail, groupby_columns=['TOD'], summary_column='count')
rail_df = rail_df.rename(columns={'TOD': 'Period', 'count': 'Value'})

In [ ]:
rail_df['Concept_ID'] = concept_id
rail_df['Metric_ID'] = 'G1.1'
rail_df['Metric_name'] = ''
rail_df['Submetric'] = 'G1.1.1'
rail_df['Description'] = 'Linked Rail Trips'
rail_df['Population'] = 'Whole Population'
rail_df['Geography'] = 'Regional'
rail_df['Zone_ID'] = ''
rail_df['Origin_zone'] = ''
rail_df['Dest_zone'] = ''
rail_df['Purpose'] = ''
rail_df['Mode'] = ''
rail_df['Income'] = ''
rail_df['Units'] = 'trips'
rail_df['Total_Increment'] = ''

In [ ]:
rail_df = rail_df[perf_measure_columns]
rail_df.to_csv(_join(summary_outputs, 'G1.1.1' + '_weekday_linked_rail_trips_' + concept_id + '_region' + filename_extension + '.csv'), index=False)

In [ ]:
rail_df

In [ ]:
rail_df['Value'] = rail_df['Value'] * params['annual_transit_factor']
rail_df['Submetric'] = 'G1.1.2'

In [ ]:
rail_df.to_csv(_join(summary_outputs, 'G1.1.2' + '_annual_linked_rail_trips_' + concept_id + '_region' + filename_extension + '.csv'), index=False)

In [ ]:
# daily = rail.groupby('TOD').sum()